In [ ]:
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.interpolate import griddata 
import torch
from util_nets import *
from scipy import interpolate

cuda = torch.cuda.is_available()
device = 'cpu'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from scipy.interpolate import griddata


def grid_contour(value,coordinates_orig,limit,cmp,vml,fs1=30,cylinder_num=1,saveif=0,filename=None,bar=True):


  xmax,xmin,ymax,ymin=limit[0],limit[1],limit[2],limit[3]


  dx, dy = 0.005, 0.005
  Nx = int((xmax-xmin)/dx+1)
  Ny = int((ymax-ymin)/dy+1)
  x = np.linspace(xmin, xmax, Nx)
  y = np.linspace(ymin, ymax, Ny)
  x_mesh, y_mesh = np.meshgrid(x, y)


  grid1 = griddata(coordinates_orig,value,(x_mesh,y_mesh), method='linear', fill_value=0)


  ddx = 0.5*(x[2]-x[1])
  xca = np.linspace(x[0]-ddx, x[len(x)-1]+ddx, len(x)+1)
  ddy = 0.5*(y[2]-y[1])
  yca = np.linspace(y[0]-ddy, y[len(y)-1]+ddy, len(y)+1)
  # vim = [-0.2,1.4]


  fit = plt.figure(dpi=300,figsize=(4,2))
  ax = plt.axes()

  plt.gca().set_aspect('equal')
  plt.axis([xmin,xmax,ymin,ymax])
  plt.xticks([])
  plt.yticks([])

  if cmp==None:
    cmp='viridis'  # 'jet'

  if vml==None:
    plt.pcolor(xca,yca,grid1[:,:],cmap=cmp)
  else:
    plt.pcolor(xca,yca,grid1[:,:],cmap=cmp,vmin=vml[0],vmax=vml[1])
  plt.xticks(fontsize=fs1-5); plt.yticks(fontsize=fs1-5)  
  if cylinder_num==1:
    r = 0.05
    rx, ry = 0.2, 0.2
    cx11=np.linspace(rx-r,rx+r,100)
    cy11=np.sqrt(r*r-(cx11-rx)**2)+ry
    cx12=np.linspace(rx+r,rx-r,100)
    cy12=-np.sqrt(r*r-(cx12-rx)**2)+ry
    circx=np.concatenate([cx11,cx12])
    circy=np.concatenate([cy11,cy12])
    plt.plot(circx,circy,color='black',linewidth=0.1)
    c = patches.Circle(xy=(0.2, 0.2), radius=0.05, fc='grey', ec='k')
    ax.add_patch(c)
  elif cylinder_num==3:
    r=0.5
    rx, ry = 4.23, 0.8
    cx11=np.linspace(rx-r,rx+r,100)
    cy11=np.sqrt(r*r-(cx11-rx)**2)+ry
    cx12=np.linspace(rx+r,rx-r,100)
    cy12=-np.sqrt(r*r-(cx12-rx)**2)+ry
    circx=np.concatenate([cx11,cx12])
    circy=np.concatenate([cy11,cy12])
    plt.plot(circx,circy,color='black',linewidth=1)
    c1 = patches.Circle(xy=(rx, ry), radius=r, fc='grey', ec='k')
    ax.add_patch(c1)
    rx, ry = 4.23, -0.8
    cx21=np.linspace(rx-r,rx+r,100)
    cy21=np.sqrt(r*r-(cx21-rx)**2)+ry
    cx22=np.linspace(rx+r,rx-r,100)
    cy22=-np.sqrt(r*r-(cx22-rx)**2)+ry
    circx=np.concatenate([cx21,cx22])
    circy=np.concatenate([cy21,cy22])
    plt.plot(circx,circy,color='black',linewidth=1)
    c2 = patches.Circle(xy=(rx, ry), radius=r, fc='grey', ec='k')
    ax.add_patch(c2)
    rx, ry = 3.0, 0.0
    cx31=np.linspace(rx-r,rx+r,100)
    cy31=np.sqrt(r*r-(cx31-rx)**2)+ry
    cx32=np.linspace(rx+r,rx-r,100)
    cy32=-np.sqrt(r*r-(cx32-rx)**2)+ry
    circx=np.concatenate([cx31,cx32])
    circy=np.concatenate([cy31,cy32])
    plt.plot(circx,circy,color='black',linewidth=1)
    c3 = patches.Circle(xy=(rx, ry), radius=r, fc='grey', ec='k')
    ax.add_patch(c3)
  elif cylinder_num==0:
    plt.xticks(fontsize=fs1-5); plt.yticks(fontsize=fs1-5)
  if bar:
    plt.colorbar(fraction=0.1,aspect=50,pad=0.05,orientation='horizontal')
  if saveif:
    plt.savefig(filename,bbox_inches='tight')
  plt.show()
  return


# Load Data

In [ ]:
coor = np.load('/root/data1/P-TLDINets/lock-exchange/data/' + 'coor.npy')
# coor = np.load('/root/data1/lock-exchange/data/coor.npy')
[xmin, ymin] = np.min(coor, axis=0)
[xmax, ymax] = np.max(coor, axis=0)
limit = [xmax,xmin,ymax,ymin]
print(limit)

date_str = '04_02_2024_24_52'
date_str = '04_03_2024_26_38'
date_str = '04_04_2024_26_33save_model_test'
date_str = '04_05_2024_22_04save_model_test'


results = np.load('/root/data1/P-TLDINets/lock-exchange/results/' + date_str + '/result.npy', allow_pickle=True).item()
of_simu = np.load('/root/data1/P-TLDINets/lock-exchange/results/' + date_str + '/of_simu.npy')
s_simu = np.load('/root/data1/P-TLDINets/lock-exchange/results/' + date_str + '/s_simu.npy')
state = np.load('/root/data1/P-TLDINets/lock-exchange/results/' + date_str + '/state.npy')

print(of_simu.shape)

print(of_simu.shape)

sindy_coef = np.load('/root/data1/P-TLDINets/lock-exchange/results/' + date_str + '/sindy_coef.npy')

[0.8, 0.0, 0.1, 0.0]
(56, 150, 1491, 1)
(56, 150, 1491, 1)


In [ ]:
path_data = '/root/data1/P-TLDINets/lock-exchange/data/'
path_results = '/root/data1/P-TLDINets/lock-exchange/results/'

# data_train = np.load(path_data + 'data_train_le_441_150.npy', allow_pickle = True).item()
data = np.load(path_data + 'data_test_le_441_150_0-2.npy', allow_pickle = True).item()
# data = dataset['X_test']

problem = {
    'space': {
        'dimension' : 2 # lock-exchange problem
    },
    'input_parameters': [
        { 'name': 'coeff_velocity' },
        { 'name': 'coeff_viscosity' }
    ],
    'input_signals': [],
    'output_fields': [
        { 'name': 'Temperature' }
    ]
}

normalization = {
    'space': { 'min' : [-1], 'max' : [+1]},
    # 'time': { 'time_constant' : 0.05 },
    'input_parameters': {
        'coeff_velocity': { 'min': 5.0, 'max': 5.5 },
        'coeff_viscosity': { 'min': 1e-6, 'max': 1e-5 },
    },
    'output_fields': {
        'Temperature': { 'min': -1, 'max': +1 }
    }
}

In [ ]:
print(np.max(of_simu[:, 0, :, 0]), np.min(of_simu[:, 0, :, 0]))
print(np.max(data), np.min(data))
print(of_simu.shape)

In [ ]:


def create_dataset(data, coor, idx):  # 从data里面sample
    new_dataset = {
        'points': coor[:, :],  # (1491,2)
        'times': data['t'],   # (150,) dt=0.05
        'out_fields': data['X_test'][idx, :, :, None],
        'inp_parameters': data['param_grid'][idx, :],
        'inp_signals': None,
    }

    return new_dataset


def process_dataset(dataset, normalization_definition = None, dt = None):
    if dt is not None:
        times = np.arange(dataset['times'][0], dataset['times'][-1] + dt * 1e-10, step = dt)
        if dataset['inp_signals'] is not None:
            dataset['inp_signals'] = interpolate.interp1d(dataset['times'], dataset['inp_signals'], axis = 1)(times)
        dataset['out_fields'] = interpolate.interp1d(dataset['times'], dataset['out_fields'], axis = 1)(times)
        dataset['times'] = times
    
    num_samples = dataset['out_fields'].shape[0]
    num_times = dataset['times'].shape[0]
    num_points = dataset['points'].shape[0]
    num_x = dataset['points'].shape[1]

    points_full = np.broadcast_to(dataset['points'][None,None,:,:], [num_samples, num_times, num_points, num_x])  # (n_sample, n_time, 100, 1) 将space坐标扩展到所有时间和样本点上

    # dataset['points_full'] = points_full
    dataset['points_full'] = torch.tensor(points_full, dtype=torch.float32)
    dataset['num_points'] = dataset['points_full'].shape[2]  # s_dim
    dataset['num_times'] = num_times
    dataset['num_samples'] = num_samples

    
        
    # dataset['times']
    if normalization_definition is not None:
        dataset_normalize(dataset, problem, normalization_definition)


    dataset['inp_parameters'] = torch.tensor(dataset['inp_parameters'], dtype=torch.float32)
    dataset['out_fields'] = torch.tensor(dataset['out_fields'], dtype=torch.float32)

    


idx_train = list(range(0, 441, 8))
# idx_test =  [idx + 1 for idx in idx_train if idx + 1 <= 440]
idx_test =  list(range(0, 441, 1))
dataset_train = create_dataset(data, coor, idx_train)
dataset_test = create_dataset(data, coor, idx_test)
out_fields_normalizetion(dataset_train)
out_fields_normalizetion(dataset_test)
# process_dataset(dataset_train, dt = 0.5)
process_dataset(dataset_train, normalization)
process_dataset(dataset_test, normalization)
# process_dataset(dataset_train)
# process_dataset(dataset_test)



train_data = dataset_train['out_fields']

# Load Model

In [ ]:
import torch.nn as nn

num_latent_states = 4
coefficient_initialization = 'uniform'
poly_order = 1
library_dim = library_size(num_latent_states, poly_order)

class ResNetBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ResNetBlock, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out += residual  
        out = self.relu(out)
        return out
    
class NN(nn.Module):
    def __init__(self, problem=problem, num_latent_states=num_latent_states):
        super(NN, self).__init__()
        input_shape = (num_latent_states + len(problem['input_parameters']),)  
        # self.dyn = nn.Sequential(
        #     nn.Linear(input_shape[0], 9),
        #     nn.Tanh(),
        #     # ResNetBlock(9, 9),
        #     nn.Linear(9, 9),
        #     nn.Tanh(),
        #     # ResNetBlock(9, 9),
        #     nn.Linear(9, 9),
        #     nn.Tanh(),
        #     # ResNetBlock(9, 9),
        #     nn.Linear(9, num_latent_states)
        # )
        # input_shape = (num_latent_states + problem['space']['dimension'] + len(problem['input_parameters']),)  
        # self.rec = nn.Sequential(
        #     nn.Linear(input_shape[0], 11),
        #     nn.Tanh(),
        #     nn.Linear(11, 20),
        #     nn.Tanh(),
        #     # ResNetBlock(20, 20),
        #     nn.Linear(20, 20),
        #     nn.Tanh(),
        #     # ResNetBlock(20, 20),
        #     # nn.Linear(20, 20),
        #     # nn.Tanh(),
        #     # nn.Linear(20, 20),
        #     # nn.Tanh(),
        #     # ResNetBlock(20, 20),
        #     nn.Linear(20, 11),
        #     nn.Tanh(),
        #     nn.Linear(11, len(problem['output_fields']))
        # )
        self.dyn = nn.Sequential(
            nn.Linear(input_shape[0], 9),
            nn.Tanh(),
            # ResNetBlock(9, 9),
            # nn.Linear(9, 9),
            # nn.Tanh(),
            ResNetBlock(9, 9),
            nn.Linear(9, 9),
            nn.Tanh(),
            ResNetBlock(9, 9),
            nn.Linear(9, num_latent_states)
        )
        input_shape = (num_latent_states + problem['space']['dimension'] + len(problem['input_parameters']),)  
        self.rec = nn.Sequential(
            nn.Linear(input_shape[0], 11),
            nn.Tanh(),
            nn.Linear(11, 20),
            nn.Tanh(),
            ResNetBlock(20, 20),
            nn.Linear(20, 20),
            nn.Tanh(),
            ResNetBlock(20, 20),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            ResNetBlock(20, 20),
            nn.Linear(20, 11),
            nn.Tanh(),
            nn.Linear(11, len(problem['output_fields']))
        )
        for i in range(len(idx_train)):
            s_c = torch.nn.Parameter(torch.empty(library_dim, num_latent_states))
            if coefficient_initialization == 'uniform':
                torch.nn.init.uniform_(s_c)
            elif coefficient_initialization == 'constant':
                torch.nn.init.constant_(s_c, 0.0)
            setattr(self, f'sindy_coef_{i}', s_c)
        for i in range(len(idx_train)):
            s_c = torch.nn.Parameter(torch.empty(library_dim, num_latent_states))
            if coefficient_initialization == 'uniform':
                torch.nn.init.uniform_(s_c)
            elif coefficient_initialization == 'constant':
                torch.nn.init.constant_(s_c, 0.0)
            setattr(self, f'sindy_coef_{i}', s_c)

        state0 = torch.zeros(num_latent_states,)
        self.state0 = nn.Parameter(state0, requires_grad=True)
        torch.nn.init.constant_(self.state0, 0.0)

    # def NNdyn(self, x):
    #     x = self.dyn(x)

    # def NNrec(self, x):
    #     x = self.rec(x)

    def forward(self, x):  
        x_ds = self.dyn(x)
        x = self.rec(x_ds)
        return x


model = NN()


checkpoint_save = '/root/data1/LDNets-main/lock-exchange/result/' + date_str + "/checkpoint_last.pth"  
checkpoint = torch.load(checkpoint_save)  # 加载断点

model.load_state_dict(checkpoint['model_state_dict'])  

<All keys matched successfully>

In [9]:
def model_sindy(sindy_coef, dataset, model, poly_order,  num_latent_states):
    if hasattr(model, 'state0'):
        state0 = model.state0
        state0 = state0.cpu().detach().numpy()  # (num_latent_sates,)
        state0 = np.tile(state0, (dataset['num_samples'], 1))
    else:
        state0 = np.zeros((dataset['num_samples'], num_latent_states), dtype=np.float32)

    times = dataset['times']

    s_simu = []
    # of_simu = []

    for k in range(dataset['num_samples']):
        s_simu_k = sindy_simulate(state0[k], times, sindy_coef[k], poly_order)
        s_simu.append(s_simu_k)
    s_simu = np.stack(s_simu, axis=0)

    s_simu = torch.tensor(s_simu, dtype=torch.float32)

    s_simu_expanded = torch.unsqueeze(s_simu, dim=2) # .to(device)
    s_simu_expanded = s_simu_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], num_latent_states)

    of_simu = []
    inp_para_expanded = dataset['inp_parameters'].unsqueeze(1).unsqueeze(2)
    inp_para_expanded = inp_para_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], 2)
    input_rec = torch.cat([s_simu_expanded, dataset['points_full'], inp_para_expanded], dim=3)  # (num_sample, n_t, s_dim, 2+
    # input_rec = torch.cat([s_simu_expanded, dataset['points_full']], dim=3)  # (num_sample, n_t, s_dim, 2+4)
    for i in range(len(input_rec)):
        # print(input_rec.device)
        # aaa = model.rec(input_rec[i, :, :, :])
        # print(aaa.device)
        of_simu.append(model.rec(input_rec[i, :, :, :]).cpu().detach().numpy())
    
    of_simu = np.stack(of_simu)
    # print('of_simu', of_simu.shape)

    return s_simu, of_simu

model = model.to('cpu')
s_simu, of_simu = model_sindy(sindy_coef, dataset_train, model, poly_order,  num_latent_states)
s_simu = s_simu.cpu().detach().numpy()
print(s_simu.shape)
print(of_simu.shape)

(56, 150, 4)
(56, 150, 1491, 1)


# Draw

In [ ]:
dt = 0.05



def evolve_dynamics(dataset, model, dt): 
    # num_samples = len(dataset['inp_parameters'])
    # state = torch.zeros((dataset['num_samples'], num_latent_states), dtype=torch.float32).to(device)  # (100,2)  /  (147, 3)
    if hasattr(model, 'state0'):
        state0 = model.state0
        state0_expand = torch.unsqueeze(state0, 0)
        state = state0_expand.repeat(dataset['num_samples'], 1)
    else:
        state = torch.zeros((dataset['num_samples'], num_latent_states), dtype=torch.float32).to(device)  # (100,2)  /  (147, 3)
    
    state_history = []
    state_history.append(state)  # s(t_0).shape=(n_sample,n_z)=0
    d_state_history = []
    # dt_ref = normalization['time']['time_constant']
    dt_ref = 1

    for _ in range(dataset['num_times'] - 1):
        d_state = model.dyn(torch.cat([state, dataset['inp_parameters']], dim=-1))
        d_state_history.append(d_state)  
        state = state + dt / dt_ref * d_state   
        state_history.append(state)  
    
    d_state = model.dyn(torch.cat([state, dataset['inp_parameters']], dim=-1))
    d_state_history.append(d_state)
        
    
    ds_t = torch.stack(d_state_history).permute(1, 0, 2)
    s_t = torch.stack(state_history).permute(1, 0, 2)
    
    # return torch.stack(state_history).permute(1, 0, 2)  # (n_sample,n_t,n_latent_state) s(t)
    return ds_t, s_t


def reconstruct_output(dataset, states, model):
    states_expanded = torch.unsqueeze(states, dim=2) # .to(device)
    states_expanded = states_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], num_latent_states)
    inp_para_expanded = dataset['inp_parameters'].unsqueeze(1).unsqueeze(2)
    inp_para_expanded = inp_para_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], 2)
    # states_expanded = tf.broadcast_to(tf.expand_dims(states, axis = 2), 
    #     [dataset['num_samples'], dataset['num_times'], dataset['num_points'], num_latent_states])  # (100,101,1,2)→(n_sample100,n_t101,s_dim100,2) 在空间维度复制s_dim次
    # ccc = torch.cat([states_expanded, dataset['points_full']], dim=3)
    return model.rec(torch.cat([states_expanded, dataset['points_full'], inp_para_expanded], dim=3)) # NNrec(s(t),x)=y/z  s(t)(n_sample,n_t,2)+x(s_dim,) → (n_sample,n_t,s_dim,3) → (n_sample,n_t,s_dim,1)

def LDNet(dataset, model, dt):
    d_states, states = evolve_dynamics(dataset, model, dt)  
    return d_states, states, reconstruct_output(dataset, states, model)  # y(x,t)=(n_sample,n_t,s_dim,1)

def MSE(dataset, model, dt):
    d_states, state, out_fields = LDNet(dataset, model, dt)
    error = out_fields - dataset['out_fields']  # (n_sample, n_t, s_dim, 1)
    # print(out_fields) ×
    # print(state) √

    return d_states, state, error # torch.mean(torch.square(error))

model.eval()
with torch.no_grad():
    d_state2, state2, error2 = MSE(dataset_train, model, dt)
    d_states4, state4, out_fields4 = LDNet(dataset_train, model, dt)
state2 = state2.cpu().detach().numpy()
d_state2 = d_state2.cpu().detach().numpy()
# print(out_fields4)

In [14]:
# d_state, state2 = evolve_dynamics(dataset_train, model, dt)
# model = model.to(device)
model.eval()
with torch.no_grad():
    d_state2, state2, error2 = MSE(dataset_train, model, dt) 
state2 = state2.cpu().detach().numpy()
d_state2 = d_state2.cpu().detach().numpy()

In [ ]:
print(next(model.parameters()).device)
param_train = dataset_train['inp_parameters']
param_train = param_train.cpu().detach().numpy()
# param_train.shape
print(np.max(param_train[:, 0]), np.min(param_train[:, 0]))
print(np.max(param_train[:, 1]), np.min(param_train[:, 1]))

if len(problem.get('input_parameters', [])) > 0:  # 3>0
    inp_parameters_min = np.array([normalization['input_parameters'][v['name']]['min'] for v in problem['input_parameters']])  # (2,)
    inp_parameters_max = np.array([normalization['input_parameters'][v['name']]['max'] for v in problem['input_parameters']])  # (2,)
def normalize_back_para(v, v_min, v_max, axis = None, mode = 1, v_range = 1):
    v_min, v_max = reshape_min_max(len(v.shape), v_min, v_max, axis)
    if mode == 1:
        return 0.5 * v * (v_max - v_min)  / v_range + (v_min + v_max) 
    elif mode == 2:
        return v * (v_max- v_min) + v_min
    
param_train_back = normalize_back_para(param_train, inp_parameters_min, inp_parameters_max, axis = 1, mode=2)
param_train

print(np.max(param_train[:, 0]), np.min(param_train[:, 0]))
print(np.max(param_train[:, 1]), np.min(param_train[:, 1]))

In [ ]:
import matplotlib.pyplot as plt

t = dataset_train['times']
param_grid = dataset_train['inp_parameters']
out_fields_train = dataset_train['out_fields'].cpu().detach().numpy()

index = 0


in_train_idx = True

for index in range(0, dataset_train['num_samples']):
    # if index in idx_train:
    #     in_train_idx = True
    # else:
    #     in_train_idx = False
    print(index, param_grid[index, :], param_train_back[index, :]) # , in_train_idx)
    print(1 - np.linalg.norm((of_simu[index, :, :, 0] - out_fields_train[index,:, :, 0]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :, 0]).reshape(-1,1), ord=2), end = ' ')
    # print(1 - np.linalg.norm((uu_simu[index, :, :] - out_fields_train[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :]).reshape(-1,1), ord=2), end = ' ')
    print(1 - np.linalg.norm((of_simu[index, :, :, 0] - out_fields_train[index,:, :, 0]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :, 0]).reshape(-1,1), ord=2), end = ' ')
    print(1 - np.linalg.norm((s_simu[index, :, :] - state[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((state[index, :, :]).reshape(-1,1), ord=2))
    print(1 - np.linalg.norm((s_simu[index, :, :] - state2[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((state2[index, :, :]).reshape(-1,1), ord=2))

    # fig=plt.figure(dpi=300,figsize=(6,2))
    fig=plt.figure(dpi=300,figsize=(4,3))
    for kk in range(state.shape[-1]):
        plt.plot(t, state[index,:,kk],lw=1.0, label='r' + str(kk))
        # plt.plot(t, state2[index,:,kk], '--', label='r2' + str(kk))
        # plt.plot(t, s_simu[index,:,kk], '.-', ms=2, lw=1.0, label='r'+str(kk)+'_')
        plt.plot(t, s_simu[index,:,kk], '--', ms=2, lw=1.0, label='r'+str(kk)+'_')

    plt.yticks([-1.0, -0.5, 0.0, 0.5, 1.0], ['-1.0', '-0.5', '0.0', '0.5', '1.0'])
    plt.xticks([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0], ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0'])


    # plt.xlim([4.0,6.0])
    plt.legend(loc='lower right', fontsize=5)
    plt.show()

In [79]:
of_simu2 = of_simu

def out_fields_reverse(dataset, of_simu):
    param_grid = dataset['inp_parameters'].cpu().detach().numpy()
    out_fields_max = param_grid[:, 0]
    out_fields_min = -param_grid[:, 0]
    of_simu = (5 + of_simu) / 10 * (out_fields_max - out_fields_min)[:, np.newaxis, np.newaxis, np.newaxis]  + out_fields_min[:, np.newaxis, np.newaxis, np.newaxis]
    return of_simu

# of_simu2 = out_fields_reverse(dataset_train, of_simu2)
print(np.max(of_simu2), np.min(of_simu2))

5.0001917 -5.0261617


In [80]:
print(next(model.parameters()).device)
param_train = dataset_train['inp_parameters']
param_train = param_train.cpu().detach().numpy()
# param_train.shape
print(np.max(param_train[:, 0]), np.min(param_train[:, 0]))
print(np.max(param_train[:, 1]), np.min(param_train[:, 1]))

cpu
0.2 -0.2
1.0 0.0


In [81]:
X_train = dataset_train['out_fields']  # (55, 150, 1491, 1)
X_test = dataset_test['out_fields']  # (55, 150, 1491, 1)
X_test.shape
sindy_coef = []
for i in range(len(dataset_train['inp_parameters'])):
    sindy_coef_i = getattr(model, f'sindy_coef_{i}')
    sindy_coef.append(sindy_coef_i.cpu().detach().numpy())
print(len(sindy_coef))

state0 = model.state0
state0 = state0.cpu().detach().numpy()
print(state0.shape)

56
(4,)


In [82]:
times = dataset_train['times']
n_train = len(sindy_coef)
n_t = len(times)
n_points = dataset_train['num_points']
poly_order = 1


s_simu_train = []
    # of_simu = []

for k in range(n_train):
    s_simu_k = sindy_simulate(state0, times, sindy_coef[k], poly_order)
    s_simu_train.append(s_simu_k)
    # print(k)
print(len(s_simu_train))
s_simu_train = np.stack(s_simu_train, axis=0)  # (n_train, n_t, num_latent_states)
print(s_simu_train.shape)

56
(56, 150, 4)


In [83]:
sindy_coef_test = []
param_test = dataset_test['inp_parameters']
param_test = param_test.cpu().detach().numpy()

## knn interpolation

### predict for sindy coefficient

In [ ]:
def eval_sindy_coef(sindy_coef_train, param_train, param_test, t_grid, Z0, poly_order=1, knn=4):
    num_train = len(sindy_coef_train)  # =len(param_train)
    if num_train < 4:
        knn = 1
    
    sindy_coeff = []

    for k in range(len(param_test)):
        if knn == 1:  
            idx = np.argmin(np.linalg.norm(param_train-param_test[k], axis=1))  
            sindy_coef_k = sindy_coef_train[idx]  
        else:
            dist = np.linalg.norm(param_train-param_test[k], axis=1) 
            knn_idx = np.argsort(dist)[:knn]
            phi = np.zeros_like(knn_idx)

            if dist[knn_idx[0]] == 0: 
                phi[0] = 1  
            else:
                phi = 1 / np.linalg.norm(param_train[knn_idx]-param_test[k], axis=1)**1
            psi = phi / phi.sum()

            sindy_coef_k = np.zeros_like(sindy_coef_train[0])
            for i, kidx in enumerate(knn_idx):
                sindy_coef_k += psi[i] * sindy_coef_train[kidx]
        
        sindy_coeff.append(sindy_coef_k)

    return sindy_coeff

# sindy_coef_test = eval_sindy_coef(sindy_coef, param_train, param_test, times, state0, poly_order=1, knn=4)

In [ ]:
def eval_model(sindy_coef_train, dataset, param_train, param_test, t_grid, Z0, model, poly_order=1, knn=4):
    sindy_coef_test = eval_sindy_coef(sindy_coef_train, param_train, param_test, t_grid, Z0, poly_order, knn)
    # print(len(param_test))
    # print('sindy_coef_test', len(sindy_coef_test), sindy_coef_test[0].shape)
    s_simu = []
    of_simu = []
    for k in range(len(param_test)):  
        if len(Z0.shape) < 2: 
            s_simu_k = sindy_simulate(Z0, t_grid, sindy_coef_test[k], poly_order)
        elif len(Z0.shape) == 2:
            s_simu_k = sindy_simulate(Z0[k], t_grid, sindy_coef_test[k], poly_order)
        s_simu.append(s_simu_k)
    s_simu = np.stack(s_simu, axis=0)

    s_simu_tensor = torch.tensor(s_simu, dtype=torch.float32)

    s_simu_expanded = torch.unsqueeze(s_simu_tensor, dim=2) # .to(device)
    s_simu_expanded = s_simu_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], num_latent_states)

    of_simu = []
    inp_para_expanded = dataset['inp_parameters'].unsqueeze(1).unsqueeze(2)
    inp_para_expanded = inp_para_expanded.expand(dataset['num_samples'], dataset['num_times'], dataset['num_points'], 2)
    input_rec = torch.cat([s_simu_expanded, dataset['points_full'], inp_para_expanded], dim=3)  # (num_sample, n_t, s_dim, 2+
    # input_rec = torch.cat([s_simu_expanded, dataset['points_full']], dim=3)  # (num_sample, n_t, s_dim, 2+4)
    for i in range(len(input_rec)):
        of_simu.append(model.rec(input_rec[i, :, :, :]).cpu().detach().numpy())
    of_simu = np.stack(of_simu)

    return s_simu, of_simu, sindy_coef_test


sindy_coef_train = sindy_coef
param_test = dataset_test['inp_parameters'].cpu().detach().numpy()
s_simu_test, of_simu_test, sindy_coef_test = eval_model(sindy_coef_train, dataset_test, param_train, param_test, times, state0, model, knn=4)

##### draw test

In [ ]:
import matplotlib.pyplot as plt

t = dataset_test['times']
param_test = dataset_test['inp_parameters']
out_fields_test = dataset_test['out_fields'].cpu().detach().numpy()

index = 0


in_train_idx = True
param_test_back = normalize_back_para(param_test, inp_parameters_min, inp_parameters_max, axis = 1, mode=2)

for index in range(0, dataset_test['num_samples']):
    # if index in idx_train:
    #     in_train_idx = True
    # else:
    #     in_train_idx = False
    print(index, param_test[index, :], param_test_back[index, :]) # , in_train_idx)
    print(1 - np.linalg.norm((of_simu[index, :, :, 0] - out_fields_train[index,:, :, 0]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :, 0]).reshape(-1,1), ord=2), end = ' ')
    # print(1 - np.linalg.norm((uu_simu[index, :, :] - out_fields_train[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :]).reshape(-1,1), ord=2), end = ' ')
    print(1 - np.linalg.norm((of_simu[index, :, :, 0] - out_fields_train[index,:, :, 0]).reshape(-1,1), ord=2) / np.linalg.norm((out_fields_train[index, :, :, 0]).reshape(-1,1), ord=2), end = ' ')
    print(1 - np.linalg.norm((s_simu[index, :, :] - state[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((state[index, :, :]).reshape(-1,1), ord=2))
    print(1 - np.linalg.norm((s_simu[index, :, :] - state2[index,:, :]).reshape(-1,1), ord=2) / np.linalg.norm((state2[index, :, :]).reshape(-1,1), ord=2))

    # fig=plt.figure(dpi=300,figsize=(6,2))
    fig=plt.figure(dpi=300,figsize=(4,3))
    for kk in range(state.shape[-1]):
        plt.plot(t, state[index,:,kk],lw=1.0, label='r' + str(kk))
        # plt.plot(t, state2[index,:,kk], '--', label='r2' + str(kk))
        # plt.plot(t, s_simu[index,:,kk], '.-', ms=2, lw=1.0, label='r'+str(kk)+'_')
        plt.plot(t, s_simu[index,:,kk], '--', ms=2, lw=1.0, label='r'+str(kk)+'_')

    plt.yticks([-1.0, -0.5, 0.0, 0.5, 1.0], ['-1.0', '-0.5', '0.0', '0.5', '1.0'])
    plt.xticks([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0], ['0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0'])


    # plt.xlim([4.0,6.0])
    plt.legend(loc='lower right', fontsize=5)
    plt.show()